<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train6_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train6 = pd.read_csv(r'/content/train_data6.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train6['Target'] = train6['Target'].replace(['Others'],'Invalid')


In [ ]:
train6['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train6_Xtfidf = tfidf_vect.fit_transform(train6['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train6_Xtfidf,train6['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6747787610619469
                  precision    recall  f1-score   support

        Analysis       0.47      0.33      0.39       121
      Conclusion       0.83      0.42      0.56        48
           Facts       0.73      0.87      0.80       494
         Invalid       0.53      0.59      0.56       127
           Issue       0.66      0.51      0.57        49
Rule/Law/Holding       0.70      0.29      0.41        65

        accuracy                           0.67       904
       macro avg       0.65      0.50      0.55       904
    weighted avg       0.67      0.67      0.66       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 3, 2, 2, 1, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 3, 3, 2, 2, 2, 2, 2, 5, 2, 2, 2, 3, 2]
[0.9985330374482599, 0.9824716544726084, 0.9937426432669317, 0.998763509658836, 0.9987920420877767, 0.9959868384593377, 0.9932664517029572, 0.998591415358877, 0.9927453591064117, 0.9983269068582817, 0.9881184876614637, 0.9932811760209597, 0.9916360863665316, 0.998442088192605, 0.9957431064057262, 0.9983648360320752, 0.9982721686028625, 0.9985486891958112, 0.9980157999913792, 0.9882792140345119, 0.9962352122937732, 0.9974271400935222, 0.9969363488609387, 0.9992062818445976, 0.9821503160222316, 0.9983755612295506, 0.9827052292178765, 0.9922546409282427, 0.9947212545763421, 0.9841220815142683, 0.9924879164825235, 0.9973939515432165, 0.990907744515127, 0.9844267538009044, 0.9946988645424244, 0.9967985257326595, 0.989343137904604, 0.9861813100037946, 0.9964997835767148, 0.9982092639611078, 0.986925490380206

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [ ]:
frame_1 = [train6,unlabel_1]
train6_1 = pd.concat(frame_1)
len(train6_1)

3538

In [ ]:
x_train_1 = tfidf_vect.transform(train6_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train6_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.673728813559322
                  precision    recall  f1-score   support

        Analysis       0.44      0.32      0.37        92
      Conclusion       0.72      0.55      0.62        33
           Facts       0.73      0.89      0.80       384
         Invalid       0.55      0.50      0.52       106
           Issue       0.68      0.46      0.55        41
Rule/Law/Holding       0.63      0.33      0.43        52

        accuracy                           0.67       708
       macro avg       0.63      0.51      0.55       708
    weighted avg       0.66      0.67      0.65       708



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 3, 5, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 5, 3, 2]
[0.9987563881569076, 0.9989647579572026, 0.9954736699140273, 0.997386967581652, 0.998344233629507, 0.9979858444029401, 0.9988701690127036, 0.9959594667121073, 0.9976707644284691, 0.9966509381883089, 0.9933855282567259, 0.9970936657841762, 0.9960778103914697, 0.9965569474680889, 0.9836980899699167, 0.99893320106643, 0.998977372035585, 0.9902901046099961, 0.9989398473358414, 0.995879902558349, 0.9931318803075234, 0.9957682322512252, 0.9964311507206703, 0.9960617227780064, 0.99205515150465, 0.9945371301727223, 0.9987640876799881, 0.9860403908506038, 0.9989018940311268, 0.9988886429478848, 0.9982667688767921, 0.9986887833965239, 0.9952616278609795, 0.993649495570502, 0.996846537827632, 0.9883570289316658, 0.9975066141258665, 0.9987841033619169, 0.9989378378388434, 0.9870005905732278, 0.9976548656801653, 0.9911152592187232, 0.9980494875485366, 0.99166

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [ ]:
frame_2 = [train6_1,unlabel_2]
train6_2 = pd.concat(frame_2)
len(train6_2)

3585

In [ ]:
x_train_2 = tfidf_vect.transform(train6_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train6_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.640495867768595
                  precision    recall  f1-score   support

        Analysis       0.42      0.35      0.38       142
      Conclusion       0.63      0.42      0.51        45
           Facts       0.69      0.86      0.77       491
         Invalid       0.60      0.51      0.55       159
           Issue       0.81      0.39      0.53        67
Rule/Law/Holding       0.53      0.33      0.40        64

        accuracy                           0.64       968
       macro avg       0.61      0.48      0.52       968
    weighted avg       0.63      0.64      0.62       968



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 3, 4, 2, 2, 3, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 2]
[0.9972993422535329, 0.9979653287099595, 0.9982504407757863, 0.9966978126535877, 0.9908730666284681, 0.9956841144796301, 0.9986631256569881, 0.9984296625912835, 0.9901289364435928, 0.9985237033685408, 0.9987896084333849, 0.9984995539862511, 0.9985441551239748, 0.9916931248731009, 0.9971622158628692, 0.9867953761259006, 0.9977188451183806, 0.9979595464938064, 0.9972879497160092, 0.9984575626243611, 0.995569688188331, 0.9985263881750401, 0.996923463598293, 0.9977575702767015, 0.9982475074949078, 0.9851386096999675, 0.996647743572477, 0.9971166594935232, 0.9973417938151127, 0.9983431523359908, 0.9973683537257414, 0.9966545139476444, 0.9981175044855403, 0.9977435355002586, 0.9968276406661467, 0.9980528036825288, 0.9983302214969483, 0.9936432201722034, 0.998449458512942, 0.9803173044386699, 0.9969617499691452, 0.998

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train6_2,unlabel_3]
train6_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train6_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train6_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6652949245541838
                  precision    recall  f1-score   support

        Analysis       0.35      0.30      0.33        89
      Conclusion       0.73      0.35      0.47        46
           Facts       0.73      0.88      0.80       378
         Invalid       0.62      0.57      0.60       122
           Issue       0.71      0.46      0.56        48
Rule/Law/Holding       0.62      0.35      0.44        46

        accuracy                           0.67       729
       macro avg       0.62      0.49      0.53       729
    weighted avg       0.65      0.67      0.65       729



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 3, 2, 2, 2, 2, 3, 2, 3, 2, 5, 3, 0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9962813746244354, 0.9876416256764933, 0.9937432400075308, 0.9982790078207853, 0.9988588182649818, 0.9971726200163598, 0.9931073311275762, 0.9989524428335486, 0.9987829612796297, 0.9861212047288993, 0.9983345287743892, 0.9980189222571028, 0.9944325590820932, 0.998963685415416, 0.9987958093070572, 0.9980157935014992, 0.9984816611416845, 0.9955683161780433, 0.9866223878496133, 0.9964340465371925, 0.9988614782210777, 0.9985729369771776, 0.9985159284113151, 0.9984234380734973, 0.9986621069794834, 0.9985990147941877, 0.9982872310327119, 0.9980332507140848, 0.99851849120152, 0.9960370501752311, 0.9847433436492123, 0.986010016888203, 0.9938051849904567, 0.9952525521871868, 0.992932636338127, 0.9970953338478609, 0.9957191078739132, 0.9888565791137075, 0.9833397420683968, 0.9944012406546733, 0.994209188584756

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train6_3,unlabel_4]
train6_4 = pd.concat(frame_4)
len(train6_4)
x_train_4 = tfidf_vect.transform(train6_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train6_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6543456543456544
                  precision    recall  f1-score   support

        Analysis       0.51      0.32      0.40       155
      Conclusion       0.71      0.46      0.56        54
           Facts       0.72      0.87      0.79       526
         Invalid       0.47      0.56      0.51       140
           Issue       0.67      0.45      0.54        58
Rule/Law/Holding       0.68      0.28      0.40        68

        accuracy                           0.65      1001
       macro avg       0.63      0.49      0.53      1001
    weighted avg       0.65      0.65      0.63      1001



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 1, 2, 2, 3, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 2, 3, 3, 4, 2]
[0.9926411341617775, 0.9987912946718955, 0.9981274099281, 0.9968315621758834, 0.9980891421379994, 0.9987315343195683, 0.9979072379459532, 0.9985803244301117, 0.998357877664706, 0.9882318112443121, 0.9973166505721897, 0.9933480659895924, 0.9817774395943699, 0.9824594794585637, 0.9990846903200383, 0.9983276198048701, 0.9960082677197369, 0.9971588767572261, 0.992941083751244, 0.989111820427364, 0.9814840230026015, 0.981163209386869, 0.9959801157392572, 0.99875173224026, 0.9987413053904222, 0.9971890646892051, 0.9981274099281, 0.9970869695229607, 0.9967169968614448, 0.9876005811025059, 0.9948236961894512, 0.9985935804951811, 0.9969010904393123, 0.9990846903200383, 0.9959031783825558, 0.9982578195657211, 0.9958249645391314, 0.9980540384088302, 0.9954631367982412, 0.9800303492606719, 0.997906053928516, 0.989068853464

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train6_4,unlabel_5]
train6_5 = pd.concat(frame_5)
len(train6_5)
x_train_5 = tfidf_vect.transform(train6_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train6_5['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6509341199606686
                  precision    recall  f1-score   support

        Analysis       0.51      0.33      0.40       137
      Conclusion       0.68      0.51      0.58        55
           Facts       0.71      0.87      0.78       536
         Invalid       0.53      0.48      0.50       169
           Issue       0.56      0.41      0.47        54
Rule/Law/Holding       0.57      0.30      0.40        66

        accuracy                           0.65      1017
       macro avg       0.59      0.48      0.52      1017
    weighted avg       0.63      0.65      0.63      1017



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7178502879078695


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.700466162504466, 0.6109278564943922, 0.6435237880494326, None)
